### Python Module(s) of the Week: `folium` and `requests`
---

**folium** is a powerful library that helps you create interactive maps. By default folium creates a map in a separate HTML file. Since folium results are interactive, you can create and interact with folium maps in Jupyter notebooks. 

The **requests** library is the de facto standard for making HTTP requests in Python. It abstracts the complexities of making requests behind a beautiful, simple API so that you can focus on interacting with services and consuming data in your application.

Since neither folium nor requests are included as part of the Python standard library, we need to install them in order to use the functionality they expose. Check the *Installing Third-Party Packages* page under Module 1 in Canvas to see how to install packages from VSCode. 


In [1]:
# Once the necessary packages have been installed, import them.
import folium
import requests


## Creating Interative Maps in Folium

Creating maps with folium is straightforward. We simply pass the latitude and longitude of the point of interest (POI) and
specify a zoom level. We can then drop a circle marker on the point of interest, and interact with the map however we'd like. 

We can get the latitude and longitude for a given POI by performing a google search. Latitude ranges from -90 to 90 degrees, longitude from -180 to 180 degrees. The latitude and longitude for the DMACC Ankeny campus is **(41.5996, -93.6276)**. Note that for US coordinates, the longitude will always be negative. 


![img01](https://c.tadst.com/gfx/1200x675/longitude-and-latitude-simple.png?1)

To illustrate, let's render a map over the park I used to play at as a child (Durkin Park on the Southside of Chicago):

In [2]:

# Latitude and longitude for Durkin Park, 84th & Kolin Ave, Chicago IL. 
# Lower zoom values "zoom out": 4 = entire US; 17 = about one city block.
lat = 41.739
lon = -87.729
zoom = 17

m = folium.Map(location=[lat, lon], zoom_start=zoom)
folium.Marker(location=[lat, lon]).add_to(m)

m


We can also draw lines connecting two POIs. Next we'll draw a green line between Durkin Park and the Ankney DMACC campus. This time we'll utilize two sets of coordinates, one for Durkin Park and one for DMACC.


In [3]:

# Durkin Park coordinates.
lat0 = 41.739
lon0 = -87.729

# DMACC coordinates. 
lat1 = 41.5996
lon1 = -93.6276

# Specify zoom level.
zoom = 7

# Center map halfway between Durkin Park and DMACC.
start_lat = (lat0 + lat1) / 2
start_lon = (lon0 + lon1) / 2

# Draw map.
m = folium.Map(location=[start_lat, start_lon], zoom_start=zoom)

# Durkin Park marker. 
folium.Marker(location=[lat0, lon0], popup="Durkin Park").add_to(m)

# DMACC marker.
folium.Marker(location=[lat1, lon1], popup="DMACC").add_to(m)

# Connect POIs with purple line. 
points = [(lat0, lon0), (lat1, lon1)]
folium.PolyLine(points, color="green").add_to(m)

m


To get help or learn more about any function used above, you can simply run `function?` in an interactive cell. For example, to get more information about `folium.Marker`, run:

In [4]:
folium.Marker?

Init signature:
folium.Marker(
    location: Optional[Sequence[float]] = None,
    popup: Union[ForwardRef('Popup'), str, NoneType] = None,
    tooltip: Union[ForwardRef('Tooltip'), str, NoneType] = None,
    icon: Optional[folium.map.Icon] = None,
    draggable: bool = False,
    **kwargs: Union[str, float, bool, Sequence, dict, NoneType],
)
Docstring:     
Create a simple stock Leaflet marker on the map, with optional
popup text or Vincent visualization.

Parameters
----------
location: tuple or list
    Latitude and Longitude of Marker (Northing, Easting)
popup: string or folium.Popup, default None
    Label for the Marker; either an escaped HTML string to initialize
    folium.Popup or a folium.Popup instance.
tooltip: str or folium.Tooltip, default None
    Display a text when hovering over the object.
icon: Icon plugin
    the Icon plugin to use to render the marker.
draggable: bool, default False
    Set to True to be able to drag the marker around the map.

Returns
-------
Mark


## Requests
---

**requests** is the de facto standard for making HTTP requests in Python (downloading Wikipedia data, querying REST APIs, etc.)
An API is a set of definitions and protocols for building and integrating application software. It is a bit of an oversimplification,
but one way to think of an API is a way to expose data to entities without having to provide direct database access to an end user.



### ISS

NASA maintains the OpenNotify API, which gives real-time location information for the International Space Station. If you paste the following URL into your browser:

```
http://api.open-notify.org/iss-now.json
```

You'll get a response like the following:

```
{"timestamp": 1705184237, "message": "success", "iss_position": {"latitude": "-2.4542", "longitude": "99.3731"}}
```

The response has a status message, a timestamp and associated latitude and longitude. This is returned in JSON format, which is a common data format used in web applications (JSON is similar to the Python dictionary, which we'll cover later). Here is how we can request data from OpenNotify using requests:


In [10]:

# Address for OpenNotify API.
url  = "http://api.open-notify.org/iss-now.json"

# Retrieve current position of ISS as JSON.
dresp = requests.get(url).json()

# Unpack/flatten results.
dpos = {
    "timestamp": dresp["timestamp"],
    "latitude": float(dresp["iss_position"]["latitude"]),
    "longitude": float(resp["iss_position"]["longitude"])
    }




{'timestamp': 1705184783, 'latitude': -29.1238, 'longitude': 115.1533}

The `dresp` object is returned as a Python dictionary. We unpack the results and convert latitude and longitude to floating point types since the API returns them as strings. 

We can put this code into a function so we can call it repeatedly without having to copy the same code at each invocation:

In [5]:

def get_iss():
    """
    Get position at specified time of ISS.

    Returns
    -------
    dict
    """
    url  = "http://api.open-notify.org/iss-now.json"

    # Retrieve current position of ISS as JSON.
    dresp = requests.get(url).json()

    # Unpack/flatten results.
    dpos = {
        "timestamp": dresp["timestamp"],
        "latitude": float(dresp["iss_position"]["latitude"]),
        "longitude": float(dresp["iss_position"]["longitude"])
        }
    
    return dpos


get_iss()



{'timestamp': 1705185955, 'latitude': -45.7616, 'longitude': -144.2722}

### Combining requests with folium

We can use the functionality exposed by folium and requests to visualize how the position of the International Space Station changes over time.
The steps are as follows:

- Collect ISS lat/lon pairs every minute for 10 minutes. We will hold the dictionaries returned from `get_iss` in a list. 

- Once with have the dictionaries, extract the latitude and longitude from each observation into a list of lists, which we can then pass into folium.

- Create a map visualizing the trajectory of the ISS with a marker for each observation. 



In [10]:

from pprint import pprint
import time


# List to hold ISS location at each invocation.
positions = []

# Number of observations to make.
nbr_obs = 20

# Time in seconds to sleep in between requests. 
time_to_sleep = 60


for i in range(nbr_obs):

    # Print status update.
    print(f"Observation #{i}.")

    # Query current ISS position. 
    curr_pos = get_iss()

    # Append curr_pos dict to positions list. 
    positions.append(curr_pos)

    # Sleep for time_to_sleep seconds before next position request.
    time.sleep(time_to_sleep)


pprint(positions, width=90)

Observation #0.
Observation #1.
Observation #2.
Observation #3.
Observation #4.
Observation #5.
Observation #6.
Observation #7.
Observation #8.
Observation #9.
Observation #10.
Observation #11.
Observation #12.
Observation #13.
Observation #14.
Observation #15.
Observation #16.
Observation #17.
Observation #18.
Observation #19.
[{'latitude': -40.5225, 'longitude': -133.119, 'timestamp': 1705186112},
 {'latitude': -38.2154, 'longitude': -129.3451, 'timestamp': 1705186172},
 {'latitude': -35.7788, 'longitude': -125.825, 'timestamp': 1705186232},
 {'latitude': -33.2313, 'longitude': -122.5339, 'timestamp': 1705186292},
 {'latitude': -30.5885, 'longitude': -119.4468, 'timestamp': 1705186352},
 {'latitude': -27.8642, 'longitude': -116.539, 'timestamp': 1705186412},
 {'latitude': -25.0478, 'longitude': -113.7658, 'timestamp': 1705186473},
 {'latitude': -22.196, 'longitude': -111.1507, 'timestamp': 1705186533},
 {'latitude': -19.2957, 'longitude': -108.6506, 'timestamp': 1705186593},
 {'latit

Finally, we can extract the latitude and longitude of our 1 minute spaced observations and plot them using `PolyLine`. 

In [20]:

# Specify zoom level.
zoom = 7

# Create list of lat-lon pairs to pass into folium.
latlons = [(dd["latitude"], dd["longitude"]) for dd in positions]

# Draw map.
m = folium.Map(location=latlons[10], zoom_start=zoom)

# Draw marker for each lat-lon pair.
for ll in latlons:

    folium.CircleMarker(
        location=ll, radius=5, color="red", fill_color="red", fill=True ,fill_opacity=1
        ).add_to(m)
    
# Draw line through points.
folium.PolyLine(latlons, color="green").add_to(m)

# # DMACC marker.
# folium.Marker(location=[lat1, lon1], popup="DMACC").add_to(m)

# # Connect POIs with purple line. 
# points = [(lat0, lon0), (lat1, lon1)]
# folium.PolyLine(points, color="green").add_to(m)

m






### Challenges:

- How can we modify the code in the last cell to add a popup with the timestamp of the associated lat-lon pair?

- Try drawing a rectangle that encloses all 20 GPS points (see `folium.Rectangle`). How would you
determine the size/extent of the rectangle?

- Calculate the distance between the first and last ISS coordinates (hint: see the [haversine formula](https://www.movable-type.co.uk/scripts/latlong.html)).

- Estimate the average speed of the ISS using the distance / time approximation. Is the speed constant over the observation period?

- Try using a different API to determine the nearest country for each GPS capture. 




### References:

- folium: https://python-visualization.github.io/folium/latest/getting_started.html
- requests: https://requests.readthedocs.io/en/latest/
- OpenNotify API: http://open-notify.org/Open-Notify-API/ISS-Location-Now/
